# 🎯 The Black Friday Playbook: Building Your Peak Event Intelligence System
## AWS re:Invent 2025 - DAT409
## Implement hybrid search with Aurora PostgreSQL for MCP retrieval

### Your Mission
Transform a year of engineering observations into actionable intelligence for Black Friday. Build a hybrid search system that surfaces hidden patterns from past incidents and creates proactive monitoring strategies.

### The Real-World Problem We're Solving
**Every engineering team describes the same problem differently:**
- **DBA**: "Noticed autovacuum process holding locks longer than usual"
- **SRE**: "Application response times degraded, seeing timeouts"
- **Developer**: "Getting connection pool exhausted exceptions"
- **Data Engineer**: "ETL jobs failing due to database unavailability"

**These are all symptoms of the same underlying issue!** Traditional search misses these connections. Today, you'll build a system that finds them all.

### What You'll Learn
1. **PostgreSQL Full-Text Search with pg_trgm**: Find variations, typos, and partial matches
2. **Semantic Search with pgvector**: Understand conceptual similarity
3. **Hybrid Search**: Combine both for comprehensive pattern discovery
4. **Cohere Reranking via Amazon Bedrock**: Improve result relevance
5. **MCP Patterns**: Structure retrieval for different contexts
6. **Search Strategy Optimization**: Choose the right approach for each query type

### Prerequisites
- AWS Account with Bedrock access
- Basic knowledge of PostgreSQL and Python
- Laptop with Jupyter environment

## The Challenge: Why Different Personas Matter

Your e-commerce platform handled 10 million transactions last Black Friday. You have invaluable data from:
- **4 engineering teams**: DBA, SRE, Developer, Data Engineer
- **365 days** of operational observations
- **50+ documented incidents** with different severity levels
- **1000+ metric anomalies** captured in logs

### Why Persona-Based Search Matters

**Different teams have different visibility:**
- **DBAs** see database internals: vacuum processes, lock contention, buffer cache metrics
- **SREs** see service health: response times, error rates, availability metrics
- **Developers** see application behavior: exceptions, query patterns, connection issues
- **Data Engineers** see pipeline health: ETL latency, data freshness, processing backlogs

**The same incident appears differently to each team:**
- DBA logs: "autovacuum process taking unusually long on orders table"
- SRE alerts: "p99 latency spike to 2.3 seconds"
- Developer logs: "ConnectionPoolExhaustedException in checkout service"
- Data Engineer reports: "Order analytics pipeline delayed by 45 minutes"

**Your search system must connect these dots to prevent future incidents.**

## Module 1: Understanding Your Historical Data

### Why This Matters
Before building search capabilities, we need to understand the structure and diversity of our data. This exploration reveals:
- How different teams document issues
- The language patterns unique to each persona
- The metadata that enables contextual filtering

Let's start by exploring the engineering wisdom hidden in your logs.

In [ ]:
# Package Installation and Setup
# Why: We need specific libraries for vector operations, PostgreSQL connectivity, and AWS integration
# The warning suppression keeps the output clean for workshop participants

import warnings
import os

# Suppress all warnings for cleaner workshop experience
warnings.filterwarnings('ignore')
os.environ['PYTHONWARNINGS'] = 'ignore'

# Core packages needed:
# - psycopg: Modern PostgreSQL adapter with async support
# - pgvector: Enables vector similarity search in PostgreSQL
# - boto3: AWS SDK for Bedrock integration (embeddings & reranking)
# - pandas/numpy: Data manipulation and numerical operations
# - tqdm: Progress bars for long-running operations
# - python-dotenv: Environment variable management

%pip install --quiet --upgrade psycopg pgvector boto3 pandas numpy tqdm python-dotenv 2>/dev/null

print("✅ Core packages installed")
print("ℹ️ Note: Any dependency warnings have been suppressed for a cleaner workshop experience")

# Quick verification that key packages are importable
try:
    import pandas as pd
    import numpy as np
    import psycopg
    print("✅ Package imports verified - ready to proceed!")
except ImportError as e:
    print(f"⚠️ Please restart kernel if you see import errors: {e}")

### Data Exploration: Understanding Your Year of Engineering Wisdom
### Why This Matters:
- Each persona captures different aspects of system health
- Understanding data distribution helps optimize search strategies
- Severity levels indicate which patterns are most critical to find

In [ ]:

import json
import warnings
warnings.filterwarnings('ignore')

# Load historical incident data from your engineering teams
# This represents a full year of observations across all teams
with open('../data/incident_logs.json', 'r') as f:
    incident_logs = json.load(f)

print(f"📊 Total logs to analyze: {len(incident_logs)}")
print(f"📅 Time range: 1 year of operational data")

# Examine the MCP-style structure
# Notice how each log has:
# - content: The actual observation text
# - mcp_metadata: Structured context for filtering and correlation
sample_log = incident_logs[0]
print("\n📋 Sample log structure:")
print(json.dumps(sample_log, indent=2)[:600] + "...")

# Analyze distribution by persona and severity
# This reveals:
# - Which teams are most active in logging (visibility gaps?)
# - Severity distribution (are we capturing enough warning signs?)
personas = {}
severities = {}
for log in incident_logs:
    persona = log['mcp_metadata']['persona']
    severity = log['mcp_metadata'].get('severity', 'info')
    personas[persona] = personas.get(persona, 0) + 1
    severities[severity] = severities.get(severity, 0) + 1

print("\n👥 Logs by engineering team:")
print("(Different teams have different observability - this affects search results)")
for persona, count in sorted(personas.items()):
    print(f"  - {persona}: {count} observations")

print("\n⚠️ Logs by severity:")
print("(Most issues start as 'info' before escalating - early detection is key)")
for severity, count in sorted(severities.items()):
    print(f"  - {severity}: {count} incidents")

## Module 2: Database Setup with Aurora PostgreSQL

### Why Aurora PostgreSQL?
- **Native pgvector support**: No external vector databases needed
- **Full-text search built-in**: Combines with vectors in the same query
- **ACID compliance**: Critical for production incident tracking
- **Managed service**: Focus on search logic, not database operations

We'll enable both pgvector and pg_trgm extensions for comprehensive search capabilities.

### Database Connection Setup
### Why This Configuration:
- psycopg: Modern PostgreSQL adapter with better performance than psycopg2
- register_vector: Enables pgvector types in Python
- autocommit=True: Simplifies DDL operations in workshop environment

In [ ]:
import psycopg
from pgvector.psycopg import register_vector
import os
from dotenv import load_dotenv

# Load environment variables for secure credential management
load_dotenv()

# Production Best Practice: Never hardcode credentials
# Workshop Studio provides these via environment variables
DB_HOST = os.getenv('DB_HOST', 'your-aurora-cluster.amazonaws.com')
DB_NAME = os.getenv('DB_NAME', 'workshop')
DB_USER = os.getenv('DB_USER', 'postgres')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'your-password')
DB_PORT = os.getenv('DB_PORT', 5432)

def get_db_connection():
    """Create a connection to Aurora PostgreSQL with vector support
    
    Production considerations:
    - Use connection pooling for multiple concurrent searches
    - Implement retry logic for transient failures
    - Monitor connection metrics for capacity planning
    """
    try:
        conn = psycopg.connect(
            host=DB_HOST,
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            port=DB_PORT,
            autocommit=True  # Simplifies workshop operations
        )
        # Enable pgvector type handling in Python
        register_vector(conn)
        print("✅ Connected to Aurora PostgreSQL")
        return conn
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        print("📝 Using local PostgreSQL for demonstration")
        # Fallback for local testing
        conn = psycopg.connect(
            host='localhost',
            dbname='workshop',
            user='postgres',
            password='postgres',
            autocommit=True
        )
        register_vector(conn)
        return conn

# Establish connection
conn = get_db_connection()

### Database Schema Creation
### Why This Schema Design:
- Combines structured data (persona, timestamp) with unstructured (content)
- Supports both vector embeddings and text search in one table
- JSONB for flexible metrics storage without schema changes
- Arrays for multi-valued attributes (related_systems)

In [ ]:
def setup_database(conn):
    """Create database schema optimized for hybrid search
    
    Key design decisions:
    - Single table design for simplicity and performance
    - 1024-dimension vectors (Cohere's embedding size)
    - JSONB for schema-free metrics (different teams track different metrics)
    - Temporal markers for time-based pattern analysis
    """
    
    with conn.cursor() as cur:
        # Enable required extensions
        print("🔧 Enabling PostgreSQL extensions...")
        
        # pgvector: Enables vector similarity search
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        
        # pg_trgm: Enables trigram-based fuzzy text matching
        cur.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm;")
        
        # Clean slate for workshop
        cur.execute("DROP TABLE IF EXISTS incident_logs CASCADE;")
        
        # Create main table with hybrid search capabilities
        print("📊 Creating incident_logs table...")
        cur.execute("""
            CREATE TABLE incident_logs (
                -- Primary identifier
                doc_id TEXT PRIMARY KEY,
                
                -- Core content for search
                content TEXT NOT NULL,
                
                -- MCP-style structured metadata
                persona TEXT NOT NULL,        -- Which team observed this?
                timestamp TIMESTAMPTZ NOT NULL, -- When did it happen?
                task_context TEXT,             -- What were they doing?
                severity TEXT DEFAULT 'info',  -- How critical was it?
                
                -- Flexible metric storage
                metrics JSONB,                 -- Team-specific metrics
                
                -- Relationship tracking
                related_systems TEXT[],        -- Which systems were involved?
                temporal_marker TEXT,          -- Time-of-day patterns
                
                -- Vector embedding for semantic search
                content_embedding vector(1024), -- Cohere embedding dimension
                
                -- Audit trail
                created_at TIMESTAMPTZ DEFAULT NOW()
            );
        """)
        
        print("✅ Database schema created successfully")

# Setup the database
setup_database(conn)

## Module 3: Understanding PostgreSQL Full-Text Search with pg_trgm

### What are Trigrams and Why Do They Matter?

**The Problem:** Engineers don't always use consistent terminology:
- One team writes "database", another writes "db"
- Typos happen under pressure: "performence" instead of "performance"
- Abbreviations vary: "conn pool" vs "connection pool"

**The Solution:** Trigrams break text into 3-character chunks, enabling fuzzy matching:
- 'database' → ['dat', 'ata', 'tab', 'aba', 'bas', 'ase']
- 'databse' (typo) → ['dat', 'ata', 'tab', 'abs', 'bse']
- These share many trigrams, so they match!

This helps find:
- **Partial matches**: "db" matches "database"
- **Typos and misspellings**: Critical during incident response
- **Similar words**: "connection" and "connections"

### Demonstrating Trigram Search Capabilities
### Why This Matters for Operations:
- During incidents, engineers make typos
- Different teams use different abbreviations
- Historical searches need to be forgiving

In [ ]:
with conn.cursor() as cur:
    # Visualize how trigrams work
    cur.execute("SELECT show_trgm('database performance') as trigrams;")
    trigrams = cur.fetchone()[0]
    print("📝 Trigrams for 'database performance':")
    print(f"   {trigrams}")
    print("   Note: Spaces become part of trigrams, enabling phrase matching\n")
    
    # Real-world similarity scoring examples
    test_phrases = [
        ('database performance', 'database performance', 'Exact match'),
        ('database performance', 'db performance', 'Common abbreviation'),
        ('database performance', 'database perf', 'Truncated term'),
        ('database performance', 'databse performence', 'Multiple typos'),
        ('database performance', 'query latency', 'Different concept')
    ]
    
    print("🔍 Similarity Scores (0 = no match, 1 = perfect match):")
    print("-" * 60)
    
    for phrase1, phrase2, description in test_phrases:
        cur.execute("SELECT similarity(%s, %s) as score;", (phrase1, phrase2))
        score = cur.fetchone()[0]
        print(f"{description:20} | '{phrase2:25}' | Score: {score:.3f}")
        
        # Explain the implications
        if score > 0.3:
            print(f"                     ✓ Would be found with fuzzy search")
        elif score > 0:
            print(f"                     ⚠ Weak match, might need lower threshold")
        else:
            print(f"                     ✗ No trigram overlap - need semantic search")

print("\n💡 Key Insight: Trigrams handle variations but miss conceptual similarity")
print("   That's why we need semantic search too!")

## Module 4: Semantic Search with pgvector and Cohere Embeddings

### How Semantic Search Complements Trigrams

**Where Trigrams Fall Short:**
- "Connection pool exhausted" and "max threads reached" have no text overlap
- "Database slow" and "query latency high" are conceptually similar but textually different

**How Semantic Search Works:**
1. **Text → Vector**: Convert text to high-dimensional vectors (embeddings)
2. **Similarity in Space**: Similar concepts have similar vectors
3. **Nearest Neighbors**: Find closest vectors in 1024-dimensional space

**Why Cohere Embed v3:**
- State-of-the-art performance for technical content
- Understands engineering terminology
- Differentiates between document indexing and query search

### Setting Up Semantic Search with Cohere Embeddings
### Why Cohere via Bedrock:
- No API key management (uses IAM roles)
- Consistent AWS billing and monitoring
- Low latency from within AWS regions
- Enterprise-grade SLAs

In [ ]:
import boto3

# Initialize Bedrock client for embedding generation
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'
)

def generate_embedding(text, input_type='search_document'):
    """
    Generate embeddings using Cohere Embed v3 via Amazon Bedrock
    
    Critical distinction:
    - 'search_document': Use when indexing logs (storing)
    - 'search_query': Use when searching (retrieving)
    
    This asymmetric approach improves search relevance by 5-10%
    
    Args:
        text: Input text to embed
        input_type: 'search_document' for indexing, 'search_query' for searching
    """
    # Cohere has a 2048 character limit
    # In production, implement chunking for longer texts
    if len(text) > 2048:
        text = text[:2048]
        print("⚠️ Text truncated to 2048 chars")
    
    try:
        request_body = {
            'texts': [text],
            'input_type': input_type  # Critical for search quality!
        }
        
        response = bedrock_runtime.invoke_model(
            modelId='cohere.embed-english-v3',
            contentType='application/json',
            accept='application/json',
            body=json.dumps(request_body)
        )
        
        response_body = json.loads(response['body'].read())
        embedding = response_body['embeddings'][0]
        
        return embedding
    
    except Exception as e:
        print(f"⚠️ Bedrock call failed: {e}")
        print("Using mock embeddings for demonstration")
        # Generate deterministic mock embedding for testing
        # In production, handle this error appropriately
        np.random.seed(hash(text) % 2**32)
        return np.random.randn(1024).tolist()

# Test embedding generation with a real-world example
test_text = "Database performance degradation during Black Friday peak traffic"
test_embedding = generate_embedding(test_text)

print(f"✅ Generated embedding with {len(test_embedding)} dimensions")
print(f"📊 Sample values: {test_embedding[:5]}...")
print(f"\n💡 These numbers represent the 'meaning' of the text in 1024-dimensional space")
print(f"   Similar texts will have similar numbers in similar positions")

## Module 5: Loading and Indexing Historical Data

### Why Proper Indexing Matters

**The Challenge:** 
- Processing 365 days of logs × 4 teams = thousands of documents
- Each search needs to check similarity across all documents
- Without indexes, searches take seconds instead of milliseconds

**Our Indexing Strategy:**
- **HNSW Index** for vectors: Hierarchical Navigable Small World graphs for fast approximate nearest neighbor search
- **GIN Index** for trigrams: Generalized Inverted Index for efficient text pattern matching
- **B-tree Indexes** for filters: Fast exact matches on persona, timestamp, severity

This combination enables sub-100ms searches across thousands of documents.

### Loading Historical Incident Data with Embeddings
### Why Batch Processing:
- Reduces database round trips
- Enables progress monitoring
- Allows for failure recovery (can resume from last batch)

In [ ]:
from tqdm import tqdm
from datetime import datetime

def load_incident_data(conn, incident_logs, batch_size=10):
    """
    Load incident logs with embeddings into database
    
    Production optimizations:
    - Parallel embedding generation (multi-threading)
    - Bulk inserts with COPY command for larger datasets
    - Checkpointing for resumable loads
    - Embedding caching to avoid regeneration
    """
    print(f"📥 Loading {len(incident_logs)} incident logs...")
    print(f"   This generates embeddings for semantic search capability")
    
    with conn.cursor() as cur:
        # Process in batches for efficiency and progress tracking
        for i in tqdm(range(0, len(incident_logs), batch_size), 
                     desc="Loading batches",
                     unit="batch"):
            batch = incident_logs[i:i+batch_size]
            
            for log in batch:
                # Generate embedding for semantic search
                # Using 'search_document' type for indexing
                embedding = generate_embedding(log['content'], 'search_document')
                
                # Insert with all metadata for comprehensive filtering
                cur.execute("""
                    INSERT INTO incident_logs (
                        doc_id, content, persona, timestamp,
                        task_context, severity, metrics,
                        related_systems, temporal_marker, content_embedding
                    ) VALUES (
                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                    ) ON CONFLICT (doc_id) DO NOTHING;
                """, (
                    log['doc_id'],
                    log['content'],
                    log['mcp_metadata']['persona'],
                    log['mcp_metadata']['timestamp'],
                    log['mcp_metadata'].get('task_context'),
                    log['mcp_metadata'].get('severity', 'info'),
                    json.dumps(log['mcp_metadata'].get('metrics', {})),
                    log['mcp_metadata'].get('related_systems', []),
                    log['mcp_metadata'].get('temporal_marker'),
                    embedding
                ))
    
    print("✅ Data loading complete")
    print("   Each log now has both text content and semantic embedding")

# Load subset for workshop speed (in production, load all)
print("📝 Note: Loading first 50 logs for workshop speed")
print("   In production, you'd load all historical data")
load_incident_data(conn, incident_logs[:50], batch_size=5)

### Creating Optimized Search Indexes
### Why Each Index Type:
- HNSW: Graph-based index for fast vector similarity (10-100x faster than brute force)
- GIN Trigram: Inverted index for fuzzy text matching
- GIN FTS: Full-text search for exact phrase matching
- B-tree: Standard indexes for filtering and sorting

In [ ]:
def create_search_indexes(conn):
    """
    Create all necessary indexes for production-grade hybrid search
    
    Index tuning parameters:
    - HNSW m=16: Number of connections per node (higher = better quality, more memory)
    - HNSW ef_construction=64: Build-time accuracy (higher = better quality, slower build)
    - GIN: Best for text pattern matching and JSONB queries
    """
    print("🔨 Creating search indexes...")
    print("   This is a one-time operation that dramatically improves search speed")
    
    with conn.cursor() as cur:
        # Vector similarity index using HNSW algorithm
        print("  Creating vector index (HNSW)...")
        print("    → Enables sub-100ms semantic searches")
        cur.execute("""
            CREATE INDEX IF NOT EXISTS idx_logs_embedding
            ON incident_logs
            USING hnsw (content_embedding vector_cosine_ops)
            WITH (m = 16, ef_construction = 64);
        """)
        
        # Trigram index for fuzzy text search
        print("  Creating trigram index...")
        print("    → Enables typo-tolerant searches")
        cur.execute("""
            CREATE INDEX IF NOT EXISTS idx_logs_content_trgm
            ON incident_logs
            USING gin(content gin_trgm_ops);
        """)
        
        # Traditional full-text search index
        print("  Creating full-text search index...")
        print("    → Enables exact phrase matching")
        cur.execute("""
            CREATE INDEX IF NOT EXISTS idx_logs_fts
            ON incident_logs
            USING gin(to_tsvector('english', content));
        """)
        
        # Indexes for filtering (critical for MCP-style contextual search)
        print("  Creating filter indexes...")
        print("    → Enables fast filtering by team, time, and severity")
        cur.execute("CREATE INDEX IF NOT EXISTS idx_logs_timestamp ON incident_logs(timestamp);")
        cur.execute("CREATE INDEX IF NOT EXISTS idx_logs_persona ON incident_logs(persona);")
        cur.execute("CREATE INDEX IF NOT EXISTS idx_logs_severity ON incident_logs(severity);")
        
        # Update table statistics for query planner
        cur.execute("ANALYZE incident_logs;")
        
    print("✅ All indexes created successfully")
    print("   Your database is now optimized for hybrid search!")

# Create the indexes
create_search_indexes(conn)

## Module 6: Implementing and Comparing Search Methods

### The Three Search Methods and When Each Excels

**1. Trigram Search (Fuzzy Matching)**
- ✅ Best for: Typos, abbreviations, partial matches
- ❌ Weakness: Misses conceptually similar content
- 📊 Example: "db perf" finds "database performance"

**2. Semantic Search (Conceptual Similarity)**
- ✅ Best for: Finding related concepts, understanding context
- ❌ Weakness: Misses exact metrics or specific terms
- 📊 Example: "slow queries" finds "high latency database operations"

**3. Full-Text Search (Exact Terms)**
- ✅ Best for: Exact phrases, specific error messages
- ❌ Weakness: Too rigid, misses variations
- 📊 Example: Only finds exact match of "ConnectionPoolExhaustedException"

Let's see them in action to understand why we need all three!

In [ ]:
# Implementing Three Search Methods
# Each method has different strengths - let's see them in action

def trigram_search(query, conn, limit=5):
    """
    Trigram-based fuzzy text search
    
    Use cases:
    - Finding logs despite typos made during incidents
    - Matching abbreviations to full terms
    - Discovering partial matches
    
    Threshold of 0.1 is lenient - adjust based on your needs
    """
    with conn.cursor() as cur:
        cur.execute("""
            SELECT
                doc_id,
                content,
                persona,
                timestamp,
                severity,
                similarity(%s, content) as score
            FROM incident_logs
            WHERE similarity(%s, content) > 0.1  -- Adjust threshold as needed
            ORDER BY score DESC
            LIMIT %s;
        """, (query, query, limit))
        
        return cur.fetchall()

def semantic_search(query, conn, limit=5):
    """
    Semantic search using vector similarity
    
    Use cases:
    - Finding conceptually related incidents
    - Discovering patterns described differently
    - Cross-team correlation (same issue, different terminology)
    
    Uses cosine distance (<=>) for similarity
    """
    # Generate query embedding with 'search_query' type
    # This is different from 'search_document' used during indexing!
    query_embedding = generate_embedding(query, input_type='search_query')
    
    with conn.cursor() as cur:
        cur.execute("""
            SELECT
                doc_id,
                content,
                persona,
                timestamp,
                severity,
                1 - (content_embedding <=> %s::vector) as score
            FROM incident_logs
            WHERE content_embedding IS NOT NULL
            ORDER BY content_embedding <=> %s::vector  -- Order by distance
            LIMIT %s;
        """, (query_embedding, query_embedding, limit))
        
        return cur.fetchall()

def fulltext_search(query, conn, limit=5):
    """
    Traditional PostgreSQL full-text search
    
    Use cases:
    - Finding exact error messages
    - Searching for specific metric names
    - Locating precise configuration values
    
    Uses stemming and stop word removal
    """
    with conn.cursor() as cur:
        cur.execute("""
            SELECT
                doc_id,
                content,
                persona,
                timestamp,
                severity,
                ts_rank_cd(to_tsvector('english', content),
                          plainto_tsquery('english', %s)) as score
            FROM incident_logs
            WHERE to_tsvector('english', content) @@ plainto_tsquery('english', %s)
            ORDER BY score DESC
            LIMIT %s;
        """, (query, query, limit))
        
        return cur.fetchall()

# Compare search methods with a real-world query
test_query = "database performance issues"
print(f"🔍 Testing query: '{test_query}'")
print(f"   This is a common search during incident investigation\n")

# Trigram search results
print("1️⃣ TRIGRAM SEARCH (Fuzzy Matching):")
print("-" * 60)
trgm_results = trigram_search(test_query, conn, limit=3)
if trgm_results:
    for doc_id, content, persona, timestamp, severity, score in trgm_results:
        print(f"[{persona}] Score: {score:.3f}")
        print(f"   {content[:100]}...\n")
else:
    print("No fuzzy matches found\n")

# Semantic search results
print("\n2️⃣ SEMANTIC SEARCH (Conceptual Similarity):")
print("-" * 60)
sem_results = semantic_search(test_query, conn, limit=3)
if sem_results:
    for doc_id, content, persona, timestamp, severity, score in sem_results:
        print(f"[{persona}] Score: {score:.3f}")
        print(f"   {content[:100]}...\n")
else:
    print("No semantic matches found\n")

# Full-text search results
print("\n3️⃣ FULL-TEXT SEARCH (Exact Terms):")
print("-" * 60)
fts_results = fulltext_search(test_query, conn, limit=3)
if fts_results:
    for doc_id, content, persona, timestamp, severity, score in fts_results:
        print(f"[{persona}] Score: {score:.3f}")
        print(f"   {content[:100]}...\n")
else:
    print("No exact matches found")
    print("   This is common - exact matching is too restrictive!")

print("\n💡 Notice how each method finds different results!")
print("   Hybrid search combines all three for comprehensive coverage.")

## Module 7: Building Hybrid Search

### The Power of Combination

**Why Hybrid Search is Essential:**

Imagine searching for "autovacuum taking too long":
- **Trigram** finds: "autovacuum process extended" (text similarity)
- **Semantic** finds: "database maintenance delays" (conceptual match)
- **Full-text** finds: Documents with exact phrase "autovacuum"

**Combining them surfaces ALL relevant incidents**, regardless of how they were described.

### Weight Tuning for Different Scenarios

- **Investigation Mode** (Semantic-heavy): Finding all related issues
- **Forensics Mode** (Keyword-heavy): Finding specific metrics or errors
- **Balanced Mode**: General-purpose search

Let's build a system that adapts to your search intent!

In [ ]:
# Building Hybrid Search
# This combines all three methods with configurable weights

def hybrid_search(query, conn, weights=None, limit=10):
    """
    Combine semantic, trigram, and full-text search with configurable weights
    
    Weight configurations for different use cases:
    
    1. Investigation Mode (semantic-heavy):
       - Use when: Exploring related incidents, pattern discovery
       - Weights: {'semantic': 0.7, 'trigram': 0.2, 'fulltext': 0.1}
    
    2. Forensics Mode (keyword-heavy):
       - Use when: Finding specific errors, exact metrics
       - Weights: {'semantic': 0.2, 'trigram': 0.1, 'fulltext': 0.7}
    
    3. Balanced Mode (default):
       - Use when: General search, unsure of exact terms
       - Weights: {'semantic': 0.4, 'trigram': 0.3, 'fulltext': 0.3}
    """
    if weights is None:
        weights = {'semantic': 0.4, 'trigram': 0.3, 'fulltext': 0.3}
    
    # Normalize weights to sum to 1.0
    total = sum(weights.values())
    weights = {k: v/total for k, v in weights.items()}
    
    # Get results from all methods (fetch more than needed for better ranking)
    semantic_results = semantic_search(query, conn, limit=20)
    trigram_results = trigram_search(query, conn, limit=20)
    fulltext_results = fulltext_search(query, conn, limit=20)
    
    # Combine scores using weighted sum
    combined_scores = {}
    
    # Process semantic results
    for doc_id, content, persona, timestamp, severity, score in semantic_results:
        combined_scores[doc_id] = {
            'content': content,
            'persona': persona,
            'timestamp': timestamp,
            'severity': severity,
            'semantic_score': score,
            'trigram_score': 0,
            'fulltext_score': 0,
            'combined_score': score * weights['semantic']
        }
    
    # Add trigram scores
    for doc_id, content, persona, timestamp, severity, score in trigram_results:
        if doc_id in combined_scores:
            combined_scores[doc_id]['trigram_score'] = score
            combined_scores[doc_id]['combined_score'] += score * weights['trigram']
        else:
            combined_scores[doc_id] = {
                'content': content,
                'persona': persona,
                'timestamp': timestamp,
                'severity': severity,
                'semantic_score': 0,
                'trigram_score': score,
                'fulltext_score': 0,
                'combined_score': score * weights['trigram']
            }
    
    # Add full-text scores
    for doc_id, content, persona, timestamp, severity, score in fulltext_results:
        # Normalize FTS score to 0-1 range
        normalized_score = min(score, 1.0)
        if doc_id in combined_scores:
            combined_scores[doc_id]['fulltext_score'] = normalized_score
            combined_scores[doc_id]['combined_score'] += normalized_score * weights['fulltext']
        else:
            combined_scores[doc_id] = {
                'content': content,
                'persona': persona,
                'timestamp': timestamp,
                'severity': severity,
                'semantic_score': 0,
                'trigram_score': 0,
                'fulltext_score': normalized_score,
                'combined_score': normalized_score * weights['fulltext']
            }
    
    # Sort by combined score
    sorted_results = sorted(
        combined_scores.items(),
        key=lambda x: x[1]['combined_score'],
        reverse=True
    )[:limit]
    
    return sorted_results

# Test hybrid search with different configurations
test_query = "connection pool problems"
print(f"🔍 Hybrid Search Query: '{test_query}'")
print(f"   A common issue that teams describe differently\n")

# Test different weight configurations
weight_configs = [
    {'semantic': 0.7, 'trigram': 0.2, 'fulltext': 0.1},  # Investigation mode
    {'semantic': 0.2, 'trigram': 0.1, 'fulltext': 0.7},  # Forensics mode
    {'semantic': 0.4, 'trigram': 0.3, 'fulltext': 0.3}   # Balanced mode
]

config_names = ['Investigation Mode (Find Related Issues)', 
                'Forensics Mode (Find Exact Terms)', 
                'Balanced Mode (General Search)']

for config_name, weights in zip(config_names, weight_configs):
    print(f"\n📊 {config_name}:")
    print(f"   Weights: Semantic={weights['semantic']:.1f}, "
          f"Trigram={weights['trigram']:.1f}, "
          f"Fulltext={weights['fulltext']:.1f}")
    print("-" * 60)
    
    results = hybrid_search(test_query, conn, weights=weights, limit=3)
    
    for doc_id, result in results:
        print(f"[{result['persona']}] Combined Score: {result['combined_score']:.3f}")
        print(f"   Component scores: Semantic={result['semantic_score']:.3f}, "
              f"Trigram={result['trigram_score']:.3f}, "
              f"FTS={result['fulltext_score']:.3f}")
        print(f"   {result['content'][:100]}...\n")

print("💡 Key Insight: Different weight configurations surface different results!")
print("   Choose weights based on your search intent.")

## Module 8: Implementing Cohere Reranking

### Why Reranking Matters

**The Problem with Simple Score Combination:**
- Scores from different methods aren't directly comparable
- A high trigram score doesn't equal a high semantic score
- Simple weighted averaging can bury the most relevant results

**How Reranking Helps:**
- Uses a specialized model trained specifically for relevance
- Understands the query-document relationship holistically
- Can boost results that match user intent, not just keywords

**Real-World Impact:**
- Improves result relevance by 20-30%
- Reduces time to find root cause during incidents
- Surfaces insights that simple scoring misses

In [ ]:
# Advanced Reranking with Cohere via Bedrock
# This adds intelligence beyond simple score combination

def rerank_results(query, search_results, limit=5):
    """
    Rerank search results using Cohere Rerank v3.5 via Bedrock
    
    Why reranking improves results:
    1. Trained on relevance: Model understands what makes a good match
    2. Context-aware: Considers the full query-document relationship
    3. Cross-encoder architecture: More accurate than embedding similarity
    
    When to use reranking:
    - User-facing search where relevance is critical
    - After hybrid search to refine top results
    - When precision matters more than recall
    """
    if not search_results:
        return []
    
    try:
        # Initialize Bedrock agent runtime for reranking
        bedrock_agent_runtime = boto3.client(
            'bedrock-agent-runtime',
            region_name='us-west-2'
        )
        
        # Prepare documents for reranking
        documents = []
        for doc_id, result in search_results:
            documents.append({
                "type": "INLINE",
                "inlineDocumentSource": {
                    "type": "TEXT",
                    "textDocument": {
                        "text": result['content']
                    }
                }
            })
        
        # Call Cohere Rerank for intelligent relevance scoring
        modelId = "cohere.rerank-v3-5:0"
        model_arn = f"arn:aws:bedrock:us-west-2::foundation-model/{modelId}"
        
        response = bedrock_agent_runtime.rerank(
            queries=[
                {
                    "type": "TEXT",
                    "textQuery": {
                        "text": query
                    }
                }
            ],
            sources=documents,
            rerankingConfiguration={
                "type": "BEDROCK_RERANKING_MODEL",
                "bedrockRerankingConfiguration": {
                    "numberOfResults": limit,
                    "modelConfiguration": {
                        "modelArn": model_arn
                    }
                }
            }
        )
        
        # Map reranked results back to original data
        reranked = []
        for result in response['results']:
            idx = result['index']
            doc_id, original_result = search_results[idx]
            reranked.append((doc_id, original_result, result['relevanceScore']))
        
        return reranked
    
    except Exception as e:
        print(f"⚠️ Reranking failed: {e}")
        print("Returning original order with mock scores")
        # Graceful degradation - return original order
        return [(doc_id, result, 1.0 - i*0.1) 
                for i, (doc_id, result) in enumerate(search_results[:limit])]

# Demonstrate the power of reranking
query = "autovacuum taking too long"
print(f"🔍 Testing Reranking for: '{query}'")
print(f"   A specific DBA concern that might be described various ways\n")

# Get hybrid search results
hybrid_results = hybrid_search(query, conn, limit=10)

print("Before Reranking (Hybrid Search):")
print("-" * 60)
for i, (doc_id, result) in enumerate(hybrid_results[:5], 1):
    print(f"{i}. [{result['persona']}] Score: {result['combined_score']:.3f}")
    print(f"   {result['content'][:80]}...\n")

# Apply intelligent reranking
reranked_results = rerank_results(query, hybrid_results, limit=5)

print("\nAfter Reranking (Cohere Intelligence):")
print("-" * 60)
for i, (doc_id, result, rerank_score) in enumerate(reranked_results, 1):
    print(f"{i}. [{result['persona']}] Rerank Score: {rerank_score:.3f}")
    print(f"   {result['content'][:80]}...\n")

print("💡 Notice: Reranking often surfaces different top results!")
print("   The model understands relevance beyond simple keyword matching.")

## Module 9: MCP-Style Structured Retrieval

### Model Context Protocol (MCP) Patterns

**What Makes MCP Different:**
- Not just searching for relevance, but building structured context
- Filters enable precise slicing of your knowledge base
- Combines semantic understanding with structured metadata

### Real-World MCP Use Cases

**1. Persona-Based Context:**
- "What did DBAs observe?" → Filter by persona='dba'
- "Show me developer exceptions" → Filter by persona='developer'

**2. Temporal Context Windows:**
- "Last week's issues" → Filter by timestamp range
- "Black Friday incidents" → Filter by specific date range

**3. Severity-Based Prioritization:**
- "Critical issues only" → Filter by severity='critical'
- "Warning signs before outage" → Filter by severity='warning'

This structured approach enables AI assistants to build rich, contextual understanding!

In [ ]:
# MCP-Style Contextual Search Implementation
# This enables structured, filter-based retrieval for AI assistants

def mcp_contextual_search(query, conn, context_filters=None, weights=None, limit=10):
    """
    MCP-style contextual search with structured filters
    
    This is how modern AI assistants retrieve context:
    1. Semantic search for conceptual relevance
    2. Structured filters for precise context
    3. Metadata for relationship understanding
    
    Filter examples for different scenarios:
    
    1. Team-specific investigation:
       context_filters={'persona': 'dba'}
       → "Show me what the database team observed"
    
    2. Time-bounded analysis:
       context_filters={'time_range': (start_date, end_date)}
       → "What happened during Black Friday?"
    
    3. Severity-based triage:
       context_filters={'severity': 'critical'}
       → "Show me only critical issues"
    
    4. Multi-filter precision:
       context_filters={'persona': 'sre', 'severity': 'warning', 
                       'time_range': (date1, date2)}
       → "SRE warnings from last week"
    """
    if weights is None:
        weights = {'semantic': 0.5, 'trigram': 0.3, 'fulltext': 0.2}
    
    # Build SQL WHERE clauses from filters
    where_clauses = []
    params = []
    
    if context_filters:
        if 'persona' in context_filters:
            where_clauses.append("persona = %s")
            params.append(context_filters['persona'])
        
        if 'time_range' in context_filters:
            start, end = context_filters['time_range']
            where_clauses.append("timestamp BETWEEN %s AND %s")
            params.extend([start, end])
        
        if 'severity' in context_filters:
            where_clauses.append("severity = %s")
            params.append(context_filters['severity'])
        
        if 'task_context' in context_filters:
            where_clauses.append("task_context = %s")
            params.append(context_filters['task_context'])
    
    where_clause = " AND ".join(where_clauses) if where_clauses else "1=1"
    
    # Generate query embedding for semantic component
    query_embedding = generate_embedding(query, input_type='search_query')
    
    combined_scores = {}
    
    with conn.cursor() as cur:
        # Semantic search with MCP filters
        cur.execute(f"""
            SELECT
                doc_id, content, persona, timestamp, severity,
                1 - (content_embedding <=> %s::vector) as score
            FROM incident_logs
            WHERE {where_clause}
                AND content_embedding IS NOT NULL
            ORDER BY content_embedding <=> %s::vector
            LIMIT 20;
        """, [query_embedding] + params + [query_embedding])
        
        for doc_id, content, persona, timestamp, severity, score in cur.fetchall():
            combined_scores[doc_id] = {
                'content': content,
                'persona': persona,
                'timestamp': timestamp,
                'severity': severity,
                'score': score * weights['semantic']
            }
        
        # Trigram search with MCP filters
        cur.execute(f"""
            SELECT
                doc_id, content, persona, timestamp, severity,
                similarity(%s, content) as score
            FROM incident_logs
            WHERE {where_clause}
                AND similarity(%s, content) > 0.1
            ORDER BY score DESC
            LIMIT 20;
        """, [query] + params + [query])
        
        for doc_id, content, persona, timestamp, severity, score in cur.fetchall():
            if doc_id in combined_scores:
                combined_scores[doc_id]['score'] += score * weights['trigram']
            else:
                combined_scores[doc_id] = {
                    'content': content,
                    'persona': persona,
                    'timestamp': timestamp,
                    'severity': severity,
                    'score': score * weights['trigram']
                }
    
    # Sort and return structured results
    sorted_results = sorted(
        combined_scores.items(),
        key=lambda x: x[1]['score'],
        reverse=True
    )[:limit]
    
    return sorted_results

# Demonstrate MCP patterns for different use cases
print("🎯 MCP-Style Contextual Search Examples")
print("   This is how AI assistants build structured context\n")

# Example 1: Team-specific observations
print("1️⃣ DBA Team Perspective on Database Issues:")
print("-" * 60)
results = mcp_contextual_search(
    "database performance problems",
    conn,
    context_filters={'persona': 'dba'},
    limit=3
)
print(f"Found {len(results)} DBA observations:")
for doc_id, result in results:
    print(f"[{result['timestamp'].strftime('%Y-%m-%d')}] Score: {result['score']:.3f}")
    print(f"   {result['content'][:100]}...\n")

# Example 2: Critical severity filter
print("\n2️⃣ Critical Incidents Only:")
print("-" * 60)
results = mcp_contextual_search(
    "outage incident failure",
    conn,
    context_filters={'severity': 'critical'},
    limit=3
)
if results:
    print(f"Found {len(results)} critical incidents:")
    for doc_id, result in results:
        print(f"[{result['persona']}] {result['timestamp'].strftime('%Y-%m-%d %H:%M')}")
        print(f"   {result['content'][:100]}...\n")
else:
    print("   No critical issues found (good news!)")
    print("   In production, you'd have more historical data")

print("\n💡 MCP Pattern Benefits:")
print("   • Precise context building for AI assistants")
print("   • Structured retrieval beyond simple relevance")
print("   • Enables persona-aware and time-aware AI responses")

## Module 10: Building Your Black Friday Playbook

### From Historical Data to Proactive Monitoring

**The Goal:** Transform past incidents into future prevention

**What We're Building:**
1. **Pattern Recognition**: Identify recurring issues across teams
2. **Early Warning Signs**: Find patterns that precede outages
3. **Monitoring Queries**: Create searches that catch issues early
4. **Team Playbooks**: Document what each team should watch for

**Why This Matters:**
- Last year's "surprise" outage had warning signs 3 days earlier
- Different teams saw different symptoms but didn't connect them
- With hybrid search, we can surface these patterns proactively

Let's mine your historical data for Black Friday insights!

In [ ]:
# Analyzing Historical Patterns for Peak Event Preparation
# This discovers what issues tend to occur and when

from datetime import datetime

def analyze_patterns(conn, time_window, min_severity='warning'):
    """
    Analyze patterns in historical data to identify early warning signs
    
    Pattern categories:
    - Database: Performance degradation patterns
    - Infrastructure: Resource exhaustion signals
    - Application: Error rate increases
    - Data Pipeline: Processing delays
    
    This helps answer:
    - What issues have we seen before?
    - Which patterns precede critical incidents?
    - What should each team monitor?
    """
    # Define pattern keywords for each category
    # These are based on common incident patterns
    patterns = {
        'database': [
            'slow query', 'high latency', 'connection pool', 
            'deadlock', 'timeout', 'lock contention', 'autovacuum'
        ],
        'infrastructure': [
            'cpu spike', 'memory pressure', 'disk space', 
            'network latency', 'load average', 'swap usage'
        ],
        'application': [
            'error rate', 'exception', 'failed request', 
            'retry storm', 'circuit breaker', 'timeout'
        ],
        'data_pipeline': [
            'etl delay', 'batch failure', 'data lag', 
            'processing backlog', 'queue depth', 'throughput'
        ]
    }
    
    findings = {}
    
    print("🔍 Searching for patterns in each category...")
    
    for category, keywords in patterns.items():
        category_issues = []
        
        # Search for each pattern
        for keyword in keywords:
            results = mcp_contextual_search(
                keyword,
                conn,
                context_filters={'time_range': time_window},
                limit=5
            )
            
            if results:
                category_issues.extend(results)
        
        # Deduplicate and keep highest scoring instances
        unique_issues = {}
        for doc_id, result in category_issues:
            if doc_id not in unique_issues or result['score'] > unique_issues[doc_id]['score']:
                unique_issues[doc_id] = result
        
        findings[category] = list(unique_issues.values())
    
    return findings

# Analyze last year's Black Friday period
print("📊 Analyzing Historical Black Friday Patterns")
print("   Looking for issues from November 20-30, 2024\n")

black_friday_2024 = (
    datetime(2024, 11, 20),  # Week before Black Friday
    datetime(2024, 11, 30)   # Day after Black Friday
)

pattern_analysis = analyze_patterns(conn, black_friday_2024)

# Display findings by category
for category, issues in pattern_analysis.items():
    if issues:
        print(f"\n🔍 {category.upper().replace('_', ' ')} PATTERNS:")
        print("-" * 60)
        
        # Show top issues with insights
        for issue in issues[:2]:  # Show top 2 per category
            severity_icon = "🔴" if issue['severity'] == 'critical' else "🟡" if issue['severity'] == 'warning' else "🟢"
            print(f"{severity_icon} [{issue['persona']}] {issue['timestamp'].strftime('%m/%d %H:%M')}")
            print(f"   {issue['content'][:100]}...")
        
        # Provide actionable insight
        if category == 'database':
            print("   → Action: Monitor query performance and connection pools")
        elif category == 'infrastructure':
            print("   → Action: Set up resource utilization alerts")
        elif category == 'application':
            print("   → Action: Implement circuit breakers and retry limits")
        elif category == 'data_pipeline':
            print("   → Action: Add pipeline lag monitoring")
        print()

print("\n💡 Key Insight: These patterns often appear 24-48 hours before major incidents!")

### Creating Proactive Monitoring Queries
### Transform discovered patterns into actionable monitoring

In [ ]:
def create_monitoring_queries(patterns):
    """
    Generate monitoring queries based on discovered patterns
    
    Creates specific queries that operations teams can use
    to detect issues before they become critical
    """
    monitoring_playbook = []
    
    # Analyze patterns to create proactive queries
    critical_patterns = {
        'autovacuum': {
            'query': 'autovacuum long running locks',
            'interval': '5 minutes',
            'threshold': 0.7,
            'team': 'DBA',
            'action': 'Check table bloat and vacuum settings'
        },
        'connection pool': {
            'query': 'connection pool exhausted timeout',
            'interval': '2 minutes',
            'threshold': 0.8,
            'team': 'SRE',
            'action': 'Scale connection pools or add read replicas'
        },
        'performance': {
            'query': 'query latency slow response degradation',
            'interval': '5 minutes',
            'threshold': 0.6,
            'team': 'Developer',
            'action': 'Review slow query log and optimize'
        },
        'etl': {
            'query': 'etl pipeline delay backlog processing',
            'interval': '10 minutes',
            'threshold': 0.7,
            'team': 'Data Engineer',
            'action': 'Check data freshness and pipeline health'
        }
    }
    
    # Extract patterns from analysis
    for category, issues in patterns.items():
        for issue in issues:
            content_lower = issue['content'].lower()
            
            # Match against critical patterns
            for pattern_key, pattern_config in critical_patterns.items():
                if pattern_key in content_lower:
                    if pattern_config not in monitoring_playbook:
                        monitoring_playbook.append(pattern_config)
    
    # Add default monitors if empty
    if not monitoring_playbook:
        monitoring_playbook = list(critical_patterns.values())[:2]
    
    return monitoring_playbook

# Generate the monitoring playbook
playbook = create_monitoring_queries(pattern_analysis)

print("📚 YOUR BLACK FRIDAY MONITORING PLAYBOOK")
print("="*60)
print("\nProactive Monitoring Strategy:")
print("Save these queries and run them continuously during peak events")
print("-" * 60)

for i, monitor in enumerate(playbook, 1):
    print(f"\n{i}. 🎯 MONITOR: {monitor['query'].upper()}")
    print(f"   Team: {monitor['team']}")
    print(f"   Check Every: {monitor['interval']}")
    print(f"   Alert if Score > {monitor['threshold']}")
    print(f"   Action: {monitor['action']}")
    
    # Provide implementation example
    print(f"\n   Implementation:")
    print(f"   ```python")
    print(f"   results = hybrid_search('{monitor['query']}', conn)")
    print(f"   if results[0][1]['combined_score'] > {monitor['threshold']}:")
    print(f"       alert_team('{monitor['team']}', results)")
    print(f"   ```")

print("\n" + "="*60)
print("💡 Pro Tip: Set up these queries in your monitoring system NOW,")
print("   before the next peak event. Prevention is better than reaction!")

## Module 11: Performance Optimization and Best Practices

### Understanding Search Performance Trade-offs

**Performance Characteristics:**
- **Trigram Search**: Fastest (~1-5ms) but limited to text similarity
- **Full-text Search**: Fast (~5-10ms) but rigid matching
- **Semantic Search**: Slower (~50-200ms) but best understanding
- **Hybrid Search**: Combines all (~100-300ms) for best results

**Optimization Strategies:**
1. **Use appropriate search for the query type** (not everything needs semantic)
2. **Cache embeddings** to avoid regeneration
3. **Tune index parameters** based on your dataset size
4. **Implement result caching** for common queries

Let's measure and optimize your search performance!

In [ ]:
# Performance Benchmarking for Production Optimization
# Understanding these metrics helps you choose the right search strategy

import time

def benchmark_search_methods(query, conn, iterations=5):
    """
    Benchmark different search methods for performance comparison
    
    Why measure performance:
    - Different queries have different performance needs
    - Real-time search needs <100ms response
    - Batch analysis can tolerate slower but more accurate search
    
    Performance expectations:
    - Interactive search: <100ms
    - Background analysis: <1000ms
    - Batch processing: Can be slower for better accuracy
    """
    methods = {
        'Trigram Search': lambda: trigram_search(query, conn),
        'Semantic Search': lambda: semantic_search(query, conn),
        'Full-text Search': lambda: fulltext_search(query, conn),
        'Hybrid Search': lambda: hybrid_search(query, conn)
    }
    
    results = {}
    
    print(f"Running {iterations} iterations per method...")
    print("(First run may be slower due to cold cache)\n")
    
    for method_name, method_func in methods.items():
        times = []
        for i in range(iterations):
            start = time.time()
            method_func()
            elapsed = time.time() - start
            times.append(elapsed * 1000)  # Convert to milliseconds
        
        results[method_name] = {
            'avg_ms': np.mean(times),
            'min_ms': np.min(times),
            'max_ms': np.max(times),
            'std_ms': np.std(times),
            'p95_ms': np.percentile(times, 95)
        }
    
    return results

# Run performance benchmarks
print("⚡ Performance Benchmarks for Production Planning\n")
query = "database performance issues"
print(f"Test Query: '{query}'")

benchmarks = benchmark_search_methods(query, conn, iterations=5)

# Display results with insights
print("\n📊 Performance Results (in milliseconds):")
print("-" * 80)
print(f"{'Method':<20} {'Avg':<10} {'Min':<10} {'Max':<10} {'P95':<10} {'Std Dev':<10}")
print("-" * 80)

for method, stats in benchmarks.items():
    # Color code based on performance
    avg_ms = stats['avg_ms']
    
    print(f"{method:<20} {avg_ms:<10.2f} {stats['min_ms']:<10.2f} "
          f"{stats['max_ms']:<10.2f} {stats['p95_ms']:<10.2f} {stats['std_ms']:<10.2f}")
    
    # Provide performance guidance
    if avg_ms < 10:
        print(f"                     ✅ Excellent for real-time search")
    elif avg_ms < 100:
        print(f"                     ✅ Good for interactive search")
    elif avg_ms < 500:
        print(f"                     ⚠️  Acceptable for background search")
    else:
        print(f"                     ⚠️  Consider optimization or caching")

# Identify optimal method
fastest = min(benchmarks.items(), key=lambda x: x[1]['avg_ms'])
most_stable = min(benchmarks.items(), key=lambda x: x[1]['std_ms'])

print(f"\n🏆 Performance Summary:")
print(f"   Fastest: {fastest[0]} ({fastest[1]['avg_ms']:.2f}ms average)")
print(f"   Most Stable: {most_stable[0]} ({most_stable[1]['std_ms']:.2f}ms std dev)")

print("\n💡 Optimization Recommendations:")
print("   1. Use Trigram for typo-tolerant autocomplete (<10ms)")
print("   2. Use Semantic for investigation and discovery")
print("   3. Use Hybrid for critical searches where accuracy matters")
print("   4. Implement caching for frequently searched queries")
print("   5. Consider async processing for non-interactive searches")

## Conclusion: Your Peak Event Intelligence System

### 🎉 Congratulations! You've Built:

**A Production-Ready Hybrid Search System** that combines:
- **PostgreSQL Trigram Search** for fuzzy matching and typo tolerance
- **pgvector Semantic Search** for conceptual understanding
- **Cohere Embeddings and Reranking** for state-of-the-art relevance
- **MCP-style Structured Retrieval** for precise context building

### 🎯 Your Black Friday Playbook Includes:

1. **Pattern Recognition**: Mine historical data for recurring issues
2. **Cross-Team Correlation**: Connect symptoms across different perspectives
3. **Proactive Monitoring**: Queries that catch issues before they escalate
4. **Optimized Search Strategies**: Right method for each query type

### 💡 Key Takeaways:

**Why Different Personas Matter:**
- DBAs see database internals others miss
- SREs catch service-level degradation
- Developers spot application-layer issues
- Data Engineers notice pipeline problems
- **Your search system connects all these perspectives!**

**Why Hybrid Search Wins:**
- No single search method handles all queries well
- Trigrams catch typos, semantic understands concepts, full-text finds exact terms
- Combining them surfaces insights impossible with any single approach

### 📚 Take-Home Resources:

- Complete hybrid search implementation
- Production-ready query templates
- Performance optimization strategies
- MCP pattern implementations

### 🚀 Next Steps:

1. **Deploy This System**: Use these patterns in your production environment
2. **Customize Weights**: Tune for your specific use cases
3. **Expand Personas**: Add more team perspectives
4. **Build Dashboards**: Visualize patterns and trends
5. **Share Knowledge**: Help other teams implement hybrid search

### 🏆 You're Now Equipped To:

- Prevent outages by finding patterns others miss
- Reduce MTTR by quickly finding relevant historical incidents
- Build AI assistants with rich, structured context
- Scale your incident knowledge across all teams

### Remember:

**"The best time to prepare for Black Friday is 364 days before Black Friday."**

Your hybrid search system makes every day a learning opportunity,
turning past incidents into future resilience.

Thank you for participating in this workshop!

**#reInvent2025 #HybridSearch #AuroraPostgreSQL #MCP**

In [ ]:
# Workshop Cleanup
print("🧹 Cleaning up resources...\n")

try:
    # Close database connection
    conn.close()
    print("✅ Database connection closed")
except:
    pass

print("\n" + "="*60)
print("\n👋 Thank you for participating in DAT409!")
print("\n🌟 Please rate this session in the re:Invent mobile app")
print("   Your feedback helps us improve future workshops")
print("\n📧 Questions? Find us at the Builder's Fair or contact:")
print("   AWS Database Specialists Team")
print("\n🚀 Now go build amazing search experiences!")
print("\n" + "="*60)